In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库

In [73]:
data = pd.read_table('train.txt',sep=' ',index_col=False) 

In [74]:
data2 = pd.read_csv('1234.csv',index_col=False) 

In [75]:
data.pop('item_category_list')
data.insert(1, 'item_category_list', data2[['item_category_list']])
data.pop('item_property_list')
data.insert(2, 'item_property_list', data2[['item_property_list']])
pass

In [76]:
data[['context_timestamp']] = data[['context_timestamp']] - 1537200000

In [77]:
# data

In [78]:
X = data[['item_category_list','item_property_list',
          'shop_review_positive_rate','shop_score_service',
         'shop_score_delivery','shop_score_description','shop_star_level'
          ,'shop_review_num_level','item_price_level','item_sales_level',
         'item_collected_level','item_pv_level',
          'user_gender_id','user_age_level',
          'user_occupation_id','user_star_level',
          'context_page_id','context_timestamp']]
Y = data[['is_trade']]
X = np.array(X)
Y = np.array(Y).ravel()

In [79]:
# 制作训练集与测试集并且进行预处理
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=666)

In [80]:
X_shop_train = X_train[:, :6]
X_item_train = X_train[:, 6:10]
X_user_train = X_train[:, 10:13]
X_content_train = X_train[:, 13:15]
X_shop_test = X_test[:, :6]
X_item_test = X_test[:, 6:10]
X_user_test = X_test[:, 10:13]
X_content_test = X_test[:, 13:15]

In [81]:
train = X_train
test = X_test

## 加载测试集

In [82]:
data2 = pd.read_table('test.txt',sep=' ',index_col=False) 

In [83]:
data3 = pd.read_csv('456.csv',index_col=False) 

In [84]:
data2.pop('item_category_list')
data2.insert(1, 'item_category_list', data3[['item_category_list']])
data2.pop('item_property_list')
data2.insert(2, 'item_property_list', data3[['item_property_list']])
pass

In [85]:
data2[['context_timestamp']] = data[['context_timestamp']] - 1537200000

In [86]:
# data3

In [87]:
X_pre = data2[['item_category_list','item_property_list',
          'shop_review_positive_rate','shop_score_service',
         'shop_score_delivery','shop_score_description','shop_star_level'
          ,'shop_review_num_level','item_price_level','item_sales_level',
         'item_collected_level','item_pv_level',
          'user_gender_id','user_age_level',
          'user_occupation_id','user_star_level',
          'context_page_id','context_timestamp']]
X_pre = np.array(X_pre,dtype='float')

## 分箱

In [88]:
#item_price_level
# bins = np.linspace(-0.5, 2.5, 4)
sha = train.shape[1]
for i in range(4,sha-1):
# i = 4
# if True:
    print((X_train[:,i][X_train[:,i]>=0].max()+1.5)-(X_train[:,i][X_train[:,i]>=0].min()-0.5))
    bins = np.arange(X_train[:,i][(X_train[:,i]>=0)].min()-0.5, X_train[:,i][X_train[:,i]>=0].max()+1.5, 1)
    bins = np.hstack(([-1.5],bins))
    which_bin = np.digitize(train[:,i].reshape(-1, 1), bins=bins)
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(which_bin)
    X_train_binned = encoder.transform(which_bin)



    which_bin = np.digitize(test[:,i].reshape(-1, 1), bins=bins)
#     encoder = OneHotEncoder(sparse=False)
#     encoder.fit(which_bin)
    X_test_binned = encoder.transform(which_bin)
    
    which_bin = np.digitize(X_pre[:,i].reshape(-1, 1), bins=bins)
#     encoder = OneHotEncoder(sparse=False)
#     encoder.fit(which_bin)
    X_pre_binned = encoder.transform(which_bin)
    print(bins)
    if (np.hstack((train,X_train_binned)).shape[1] == np.hstack((test,X_test_binned)).shape[1]):
        if (np.hstack((X_pre,X_pre_binned)).shape[1] == np.hstack((test,X_test_binned)).shape[1]):
            train = np.hstack((train,X_train_binned))
            test = np.hstack((test,X_test_binned))
            X_pre = np.hstack((X_pre,X_pre_binned))
            print("0k")
    else:
        print("No")

2.17
[-1.5   0.33  1.33  2.33]
0k
2.21333333333
[-1.5         0.28666667  1.28666667  2.28666667]
0k
23.0
[ -1.50000000e+00   4.99850000e+03   4.99950000e+03   5.00050000e+03
   5.00150000e+03   5.00250000e+03   5.00350000e+03   5.00450000e+03
   5.00550000e+03   5.00650000e+03   5.00750000e+03   5.00850000e+03
   5.00950000e+03   5.01050000e+03   5.01150000e+03   5.01250000e+03
   5.01350000e+03   5.01450000e+03   5.01550000e+03   5.01650000e+03
   5.01750000e+03   5.01850000e+03   5.01950000e+03   5.02050000e+03]
0k
27.0
[ -1.5  -0.5   0.5   1.5   2.5   3.5   4.5   5.5   6.5   7.5   8.5   9.5
  10.5  11.5  12.5  13.5  14.5  15.5  16.5  17.5  18.5  19.5  20.5  21.5
  22.5  23.5  24.5  25.5]
0k
19.0
[ -1.5  -0.5   0.5   1.5   2.5   3.5   4.5   5.5   6.5   7.5   8.5   9.5
  10.5  11.5  12.5  13.5  14.5  15.5  16.5  17.5]
0k
18.0
[ -1.5   0.5   1.5   2.5   3.5   4.5   5.5   6.5   7.5   8.5   9.5  10.5
  11.5  12.5  13.5  14.5  15.5  16.5  17.5]
0k
19.0
[ -1.5  -0.5   0.5   1.5   2.5   3.

In [104]:
bins = np.linspace(0, 86400,49)
print(bins)
bins = np.hstack(([-1.5],bins))
which_bin = np.digitize(train[:,-1].reshape(-1, 1), bins=bins)
encoder = OneHotEncoder(sparse=False)
encoder.fit(which_bin)
X_train_binned = encoder.transform(which_bin)



which_bin = np.digitize(test[:,-1].reshape(-1, 1), bins=bins)
# encoder = OneHotEncoder(sparse=False)
# encoder.fit(which_bin)
X_test_binned = encoder.transform(which_bin)

which_bin = np.digitize(X_pre[:,-1].reshape(-1, 1), bins=bins)
# encoder = OneHotEncoder(sparse=False)
# encoder.fit(which_bin)
X_pre_binned = encoder.transform(which_bin)
print(bins)
if (np.hstack((train,X_train_binned)).shape[1] == np.hstack((test,X_test_binned)).shape[1]):
    if (np.hstack((X_pre,X_pre_binned)).shape[1] == np.hstack((test,X_test_binned)).shape[1]):
        train = np.hstack((train,X_train_binned))
        test = np.hstack((test,X_test_binned))
        X_pre = np.hstack((X_pre,X_pre_binned))
        print("0k")
else:
    print("No")

[     0.   1800.   3600.   5400.   7200.   9000.  10800.  12600.  14400.
  16200.  18000.  19800.  21600.  23400.  25200.  27000.  28800.  30600.
  32400.  34200.  36000.  37800.  39600.  41400.  43200.  45000.  46800.
  48600.  50400.  52200.  54000.  55800.  57600.  59400.  61200.  63000.
  64800.  66600.  68400.  70200.  72000.  73800.  75600.  77400.  79200.
  81000.  82800.  84600.  86400.]
[ -1.50000000e+00   0.00000000e+00   1.80000000e+03   3.60000000e+03
   5.40000000e+03   7.20000000e+03   9.00000000e+03   1.08000000e+04
   1.26000000e+04   1.44000000e+04   1.62000000e+04   1.80000000e+04
   1.98000000e+04   2.16000000e+04   2.34000000e+04   2.52000000e+04
   2.70000000e+04   2.88000000e+04   3.06000000e+04   3.24000000e+04
   3.42000000e+04   3.60000000e+04   3.78000000e+04   3.96000000e+04
   4.14000000e+04   4.32000000e+04   4.50000000e+04   4.68000000e+04
   4.86000000e+04   5.04000000e+04   5.22000000e+04   5.40000000e+04
   5.58000000e+04   5.76000000e+04   5.94000000e+

In [63]:
test.shape

(119535, 164)

In [73]:
PF = PolynomialFeatures(degree=2)
PF.fit(train)
X_train = PF.transform(train)
X_test = PF.transform(test)

In [89]:
SS = StandardScaler()
SS.fit(train)
X_train = SS.transform(train)
X_test = SS.transform(test)

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.metrics import log_loss
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

In [90]:
log_reg = LogisticRegression(C = 0.01,penalty='l1')
log_reg.fit(train, Y_train)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [91]:
log_loss(Y_train, log_reg.predict_proba(train)[:,1])

0.091083679259183314

In [92]:
log_loss(Y_test, log_reg.predict_proba(test)[:,1])

0.089388817687746161

In [93]:
log_reg.predict_proba(X_pre)[:,1]

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [ ]:
# BC = BaggingClassifier(base_estimator=LogisticRegression(C = 0.01), random_state=666, n_estimators=30)
# BC.fit(train, Y_train)

In [ ]:
# log_loss(Y_train, BC.predict_proba(train))

In [ ]:
# log_loss(Y_test, BC.predict_proba(test))

In [ ]:
# log_reg.predict_proba(train)[:,1]

In [13]:
# ETR = joblib.load('allETR.pkl')
ETR = ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=666, max_depth=500)#0.41
ETR.fit(train, Y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=500, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
           oob_score=False, random_state=666, verbose=0, warm_start=False)

In [133]:
log_loss(Y_train, ETR.predict_proba(train)[:,1])

0.091794363216569669

In [134]:
log_loss(Y_test, ETR.predict_proba(test)[:,1])

0.0899327885097429

In [377]:
# joblib.dump(ETR, 'allETR.pkl') 

In [77]:
MNB = MultinomialNB(alpha=1)
MNB.fit(train+1, Y_train)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [78]:
log_loss(Y_test, MNB.predict_proba(test+1)[:,1])

16.114825702706991

In [79]:
BNB = BernoulliNB(alpha=1)
BNB.fit(train+1, Y_train)

BernoulliNB(alpha=1, binarize=0.0, class_prior=None, fit_prior=True)

In [80]:
log_loss(Y_test, BNB.predict_proba(test)[:,1])

33.901286592858881

In [393]:
last = LogisticRegression(C = 0.0008,penalty='l2')

# log_reg.fit(train, Y_train)
# ETR.fit(train, Y_train)
# MNB.fit(train+1, Y_train)
# BNB.fit(train+1, Y_train)
c_data = log_reg.predict_proba(train)[:, 1].reshape(-1, 1)
c_data = np.hstack((c_data,ETR.predict_proba(train)[:, 1].reshape(-1, 1)))
c_data = np.hstack((c_data,MNB.predict_proba(train+1)[:, 1].reshape(-1, 1)))
# c_data = np.hstack((c_data,BNB.predict_proba(train+1)[:, 1].reshape(-1, 1)))
last.fit(c_data,Y_train)


LogisticRegression(C=0.0008, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [394]:
Y_pre1 = log_reg.predict_proba(train)[:, 1].reshape(-1, 1)
Y_pre2 = ETR.predict_proba(train)[:, 1].reshape(-1, 1)
Y_pre3 = MNB.predict_proba(train+1)[:, 1].reshape(-1, 1)
# Y_pre4 = BNB.predict_proba(train+1)[:, 1].reshape(-1, 1)
Y_add = np.hstack((Y_pre1, Y_pre2))
Y_add = np.hstack((Y_add, Y_pre3))
# Y_add = np.hstack((Y_add, Y_pre4))
Y_pre = last.predict_proba(Y_add)

In [395]:
log_loss(Y_train, Y_pre)

0.042162415180860724

In [396]:
# log_loss(Y_test, last.predict_proba(np.hstack((log_reg.predict_proba(test)[:, 1].reshape(-1, 1),ETR.predict_proba(test)[:, 1].reshape(-1, 1)))))

In [397]:
Y_pre1 = log_reg.predict_proba(test)[:, 1].reshape(-1, 1)
Y_pre2 = ETR.predict_proba(test)[:, 1].reshape(-1, 1)
Y_pre3 = MNB.predict_proba(test+1)[:, 1].reshape(-1, 1)
# Y_pre4 = BNB.predict_proba(test+1)[:, 1].reshape(-1, 1)
Y_add = np.hstack((Y_pre1, Y_pre2))
Y_add = np.hstack((Y_add, Y_pre3))
# Y_add = np.hstack((Y_add, Y_pre4))
Y_pre = last.predict_proba(Y_add)

# Y_pre = Y_pre1 + Y_pre2 + Y_pre3 + Y_pre4
# Y_pre = Y_pre/4

In [398]:
log_loss(Y_test, Y_pre)

0.094213842438789233

In [54]:
Y_pre = ETR.predict_proba(X_pre)
Y_pre

array([[ 0.99666667,  0.00333333],
       [ 0.96666667,  0.03333333],
       [ 0.99666667,  0.00333333],
       ..., 
       [ 0.99333333,  0.00666667],
       [ 0.99666667,  0.00333333],
       [ 0.99      ,  0.01      ]])

In [32]:
Y_pre = log_reg.predict_proba(X_pre)[:,1]
Y_pre = Y_pre.reshape(-1,1)
Y_prelist = Y_pre.tolist()

In [55]:
# sub = pd.DataFrame()
# sub['instance_id'] = np.array(data2[['instance_id']],dtype='float').ravel()
# sub['predicted_score'] = Y_pre[:,1].ravel()
# sub.to_csv('submission.csv',index=False)

In [55]:
# instance_id = np.array(data2[['instance_id']],dtype='float').ravel()
# instance_idlist = instance_id.tolist()
# with open('123456.txt', 'w') as f:
#     f.write("instance_id predicted_score\r") 
#     for i in range(Y_pre.shape[0]):
#         f.write(str(data2[['instance_id']].iloc[i,0]) + " " + str(Y_prelist[i][0]) + '\r') 

In [34]:
Y_pre.shape

(18371, 1)

In [45]:
# data2[['instance_id']]

In [44]:
sub = pd.DataFrame()
data2 = pd.read_table('test.txt',sep=' ',index_col=False) 
sub['instance_id'] = data2.pop('instance_id')
sub['predicted_score'] = Y_pre[:,0].ravel()
sub.to_csv('000.txt',index=False, sep=' ')

array([ 0.00775228,  0.02247293,  0.02156624, ...,  0.03197815,
        0.03737831,  0.03079344])